In [ ]:
from tkinter import N
from typing import final
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from time import sleep

fail = []
# driver = webdriver.Chrome(executable_path='./chromedriver');
def scrape_a_bird(birdname,driver):
    try:
        driver.get('https://www.wikidata.org/wiki/Wikidata:Main_Page')
        input_avibase = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Search Wikidata']")))
        input_avibase.send_keys(birdname)
        input_avibase.send_keys(Keys.ENTER)
        sleep(1)
        xp = '//*[@id="redlist-js"]/div/div/div[2]/section/div[2]/article/a'
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xp)))
            driver.find_elements(by=By.XPATH, value=xp)[0].click()
        except Exception:
            sleep(1)
            new_xp = '//*[@id="redlist-js"]/div/div/div[2]/section/div[2]/article/a'
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, new_xp)))
                driver.find_elements(by=By.XPATH, value=new_xp)[0].click()
            except Exception:
                fail.append(birdname)
        finally:        
            # sleep(3)
            xp_expand = '//*[@id="details__toggle-all"]'
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xp_expand)))
                expand = driver.find_elements(by=By.XPATH, value=xp_expand)
                expand[0].click()
            except Exception:
                fail.append(birdname+':Expandfail')
            finally:
                sleep(1)
                try:
                    xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/p[1]'
                    # xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/p'
                    new_xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/div/p'
                    xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p[1]'
                    # xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p'
                    new_xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p'
                    xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p[2]'
                    # xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p'
                    new_xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p'
                    xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p[2]'
                    # xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p'
                    new_xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p'
                    xp_sci_n_year = '//*[@id="taxonomy-details"]/div[1]/div[2]/p'
                    xp_conserv = '//*[@id="assessment-information"]/div[2]/div/p[1]/a/strong'
                    xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em[1]'
                    # xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em'
                    new_xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em'
                    xp_order = '//*[@id="taxonomy"]/div[2]/div[1]/p/a/strong'
                    xp_family = '//*[@id="taxonomy"]/div[2]/div[2]/p/a/strong'
                    xp_genus = '//*[@id="taxonomy"]/div[2]/div[3]/p/a/strong'
                    xp_other_c_names = '//*[@id="taxonomy-details"]/div[2]/div[2]/div/p'
                    sci_name = driver.find_elements(by=By.XPATH, value=xp_sci_name)
                    d={'Name':[birdname],'IUCN_link':[driver.current_url]}
                    try:
                        d['sci_name'] = [sci_name[0].text]
                    except Exception:
                        new_xp_sci = '//*[@id="taxonomy-details"]/div[1]/div[2]/p'
                        try:
                            d['sci_name'] = [driver.find_elements(by=By.XPATH, value=new_xp_sci)[0].text]
                        except Exception:
                            d['sci_name'] = np.nan
                    order = driver.find_elements(by=By.XPATH, value=xp_order)
                    try:
                        d['order'] = [order[0].text]
                    except Exception:
                        d['order']= np.nan
                    family = driver.find_elements(by=By.XPATH, value=xp_family)
                    try:
                        d['family'] = [family[0].text]
                    except Exception:
                        d['family']= np.nan
                    genus = driver.find_elements(by=By.XPATH, value=xp_genus)
                    try:
                        d['genus'] = [genus[0].text]
                    except Exception:
                        d['genus']= np.nan
                    sci_n_year = driver.find_elements(by=By.XPATH, value=xp_sci_n_year)
                    try:
                        d['sci_n_year'] = [sci_n_year[0].text]
                    except Exception:
                        d['sci_n_year']= np.nan
                    year = driver.find_elements(by=By.XPATH, value=xp_year)
                    try:
                        d['year'] = [year[0].text]
                    except Exception:
                        d['year']= np.nan
                    altitude = driver.find_elements(by=By.XPATH, value=xp_alt)
                    try:
                        d['altitude'] = [altitude[0].text]
                    except Exception:
                        d['altitude']= np.nan
                    populate = driver.find_elements(by=By.XPATH, value=xp_populate)
                    try:
                        d['populate'] = [populate[0].text]
                    except Exception:
                        d['populate']= np.nan
                    habitat = driver.find_elements(by=By.XPATH, value=xp_habitat)
                    try:
                        d['habitat'] =[habitat[0].text]
                    except Exception:
                        d['habitat']= np.nan
                    conservation =  driver.find_elements(by=By.XPATH, value=xp_conserv)
                    try:
                        d['conservation'] = [conservation[0].text]
                    except Exception:
                        d['conservation']= np.nan
                    other_names = driver.find_elements(by=By.XPATH, value=xp_other_c_names)
                    try:
                        d['other_names'] = [other_names[0].text]
                    except Exception:
                        d['other_names'] = np.nan
                    # print(d)
                    idf = pd.DataFrame(d)
                    return idf
                except Exception:
                    fail.append(birdname)
                    return pd.DataFrame(d)
    except Exception:
        fail.append(birdname)
                
def iucn(fro,to,driver,data_df,list_of_birds):
    for bird_name in list_of_birds[fro:to]:
        try:
            data_df = pd.concat([data_df,scrape_a_bird(bird_name,driver)] )
            logging.debug('[+] '+ bird_name)
            print('[+] '+ bird_name)
        except Exception:
            logging.error('Unknown Exceptions for Bird-' + bird_name)
    return data_df

# # if __name__ == '__main__':
# startTime = datetime.now()
# birds = pd.DataFrame(columns=['Name','sci_name','order','family','genus','sci_n_year','year','altitude','populate','habitat','conservation','other_names','IUCN_link'])
# # driver = webdriver.Chrome('./chromedriver.exe')
# dz = pd.read_excel("assets/DataZone/HBW-BirdLife_List_of_Birds_v6.xlsx")
# list_of_birds = list(dz.loc[:,'Common name'])
# # f = int(input('from :'))
# # t = int(input('to :'))
# f = 8888
# t = 8891
# ds = 'iucn_'+str(f)+'_'+str(t)
# logging.basicConfig(filename="assets/rescrape_iucn/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
# birds = iucn(f,t,driver,birds,list_of_birds)
# birds.to_pickle("assets/rescrape_iucn/Pick_"+ds+".pkl")
# birds.to_csv("assets/rescrape_iucn/csv_"+ds+".csv")
# logging.info('\n\nTime taken : ',datetime.now() - startTime)
# print('falied birds : ',fail)
# print('\n\nTime taken : ',datetime.now() - startTime) 

# driver.quit()    
'''
conda activate indicwiki & cd Documents\birds & python Eol_Scraper_Optim.py
<f>
<t>
'''
if __name__ == '__main__':
    startTime = datetime.now()
    birds = pd.DataFrame(columns=['Name','sci_name','order','family','genus','sci_n_year','year','altitude','populate','habitat','conservation','other_names','IUCN_link'])
    driver = webdriver.Chrome(executable_path='./chromedriver');
    dz = pd.read_excel("assets/DataZone/HBW-BirdLife_List_of_Birds_v6.xlsx")
    list_of_birds = list(dz.loc[:,'Common name'])
    ds = 'None-error'
    try:
        f = int(input('from :'))
        t = int(input('to :'))
        ds = 'iucn_'+str(f)+'_'+str(t)
        logging.basicConfig(filename="assets/rescrape_iucn/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
        birds = iucn(f,t,driver,birds,list_of_birds)
    finally:
        birds.to_pickle("assets/rescrape_iucn/Pick_"+ds+".pkl")
        birds.to_csv("assets/rescrape_iucn/csv_"+ds+".csv")
        try:
            logging.info('\n\nTime taken : ',datetime.now() - startTime)
        finally:
            if len(fail) != 0 :
                print('falied birds : ',fail)
            print('\n\nTime taken : ',datetime.now() - startTime) 
            driver.quit()    
'''
conda activate indicwiki & cd Documents\birds & python Iucn_Scraper_Optim.py
<f>
<t>
'''


'''
1 
2 
3 
4 
5 ['Pangani Longclaw', 'Pangani Longclaw:Expandfail', 'Five-striped Sparrow', 'Five-striped Sparrow:Expandfail', 'Stripe-capped Sparrow', 'Stripe-capped Sparrow:Expandfail', 'Chestnut-capped Brush-finch', 'Chestnut-capped Brush-finch:Expandfail', 'Amazonian Oropendola', 'Amazonian Oropendola:Expandfail', 'Greater Antillean Grackle', 'Greater Antillean Grackle:Expandfail', 'Hermit Warbler', 'Hermit Warbler:Expandfail', 'Scarlet-and-white Tanager', 'Scarlet-and-white Tanager:Expandfail', 'Slate-colored Seedeater', 'Slate-colored Seedeater:Expandfail', 'Buff-bellied Tanager', 'Buff-bellied Tanager:Expandfail', 'Rusty-browed Warbling-finch', 'Rusty-browed Warbling-finch:Expandfail', 'Three-striped Hemispingus', 'Three-striped Hemispingus:Expandfail', 'Ochraceous Conebill', 'Ochraceous Conebill:Expandfail', 'Rusty Flowerpiercer', 'Rusty Flowerpiercer:Expandfail', 'Scaled Flowerpiercer', 'Scaled Flowerpiercer:Expandfail', 'Black-throated Flowerpiercer', 'Black-throated Flowerpiercer:Expandfail', 'Merida Flowerpiercer', 'Merida Flowerpiercer:Expandfail', 'Grass-green Tanager', 'Grass-green Tanager:Expandfail', 'Black-capped Tanager', 'Black-capped Tanager:Expandfail']
'''

In [3]:
from tkinter import N
from typing import final
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from time import sleep


In [4]:

driver = webdriver.Chrome(executable_path='./chromedriver');
birdname = 'American Crow'
driver.get('https://www.wikidata.org/wiki/Wikidata:Main_Page')
input_wd = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Search Wikidata']")))
input_wd.send_keys(birdname)
input_wd.send_keys(Keys.ENTER)

C:\Users\LEGION\AppData\Local\Temp\ipykernel_21140\3897690117.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver');


In [10]:
flag  = False
confirm = '//*[@id="mw-content-text"]/div[4]/ul/li[1]/div[2]/span'
sea = driver.find_elements(by=By.XPATH, value=confirm)[0].text.strip()
if sea == 'species of bird':
    flag = True

In [11]:
if flag:
    first_res ='//*[@id="mw-content-text"]/div[4]/ul/li[1]/div[1]/a'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, first_res)))
    driver.find_elements(by=By.XPATH, value=first_res)[0].click()
    

In [12]:
dz = pd.read_pickle("data/Birds.pkl")
list_of_birds = list(dz.loc[:,'Bird Original Name'])


In [14]:
len(list_of_birds)

12658

In [15]:
driver.close()


In [3]:
!pip install scrapy

  Using cached Scrapy-2.6.1-py2.py3-none-any.whl (264 kB)
  Using cached Protego-0.2.1-py2.py3-none-any.whl (8.2 kB)
  Using cached PyDispatcher-2.0.5-py3-none-any.whl
  Using cached queuelib-1.6.2-py2.py3-none-any.whl (13 kB)
  Using cached itemadapter-0.5.0-py3-none-any.whl (10 kB)
  Using cached service_identity-21.1.0-py2.py3-none-any.whl (12 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached zope.interface-5.4.0-cp39-cp39-win_amd64.whl (210 kB)
  Using cached itemloaders-1.0.4-py3-none-any.whl (11 kB)
  Using cached parsel-1.6.0-py2.py3-none-any.whl (13 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached Automat-20.2.0-py2.py3-none-any.whl (31 kB)
  Using cached incremental-21.3.0-py2.py3-none-any.whl (15 kB)
  Using cached twisted_iocpsupport-1.0.2-cp39-cp39-win_amd64.whl (45 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
  Using cached hyperlink-21.0.0-p

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import scrapy
import pandas as pd
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from lxml import etree

In [ ]:

def scrape_v_3_ebird(URL):
    if URL != None :
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("div",class_="Breadcrumbs")
        names_main = soup.find_all("span",class_="Heading-main")
        names_sub = soup.find_all("span",class_="Heading-sub")
        udescription=soup.find_all("p",class_="u-stack-sm")
        # print(birds)
        for i,j,k,l in zip(results,names_main,names_sub,udescription):
            a = i.get_text().strip().split('\n')
            b = j.get_text().strip()
            c = k.get_text().strip()
            e = l.get_text()
            d = {'Common Name':[b],'Kingdom':['Animalia'],'Phylum':['Chordata'],'Class':['Aves'],'Order':[a[0]],'Family':[a[1]],'Binomial Name':[c],'Description':[e],'Link':[URL]}
            return pd.DataFrame(d) 
    emp_d = {'Common Name':[b],'Kingdom':pd.notna,'Phylum':pd.notna,'Class':pd.notna,'Order':pd.notna,'Family':pd.notna,'Binomial Name':pd.notna,'Description':pd.notna,'Link':[URL]} 
    
    return pd.DataFrame(emp_d)
    

In [9]:
Url = 'https://www.wikidata.org/wiki/Q839854'
page = requests.get(Url)
soup = BeautifulSoup(page.content, "html.parser")

In [10]:
cs_xp = '//*[@id="q1923584$719E3AD2-C229-4A30-B8B2-93ECEAEDE2EA"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/a'
//*[@id="q839854$0F0B42B0-3766-4703-9689-4F5891A28254"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/a
img_xp = '//*[@id="q839854$F18A1C35-0744-465C-A2BD-AC082E429BFB"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/div[2]/a'
dom = etree.HTML(str(soup))
print(dom.xpath(cs_xp)[0].text)
print(dom.xpath(img_xp)[0].text)

IndexError: list index out of range

In [11]:
!pip install wikidata

In [12]:
from wikidata.client import Client
client = Client()

In [14]:
Url = 'https://www.wikidata.org/wiki/Q839854'
entity = client.get(Url.split('/')[-1], load=True)

In [15]:
entity

<wikidata.entity.Entity Q839854 "Abbott's Booby">

In [16]:
entity.description

m'species of bird'

In [17]:
image_prop = client.get('P18')

In [18]:
image = entity[image_prop]

In [24]:
imagename =''
for i in str(image).split('"'):
    if 'File:' in i:
        imagename = i.split(':')[-1]
imagename

"Abbott's Booby.jpg"

In [20]:
image.image_resolution

(1018, 948)

In [21]:
image.image_url

'https://upload.wikimedia.org/wikipedia/commons/5/53/Abbott%27s_Booby.jpg'

In [25]:
image.title

"File:Abbott's Booby.jpg"

In [26]:
wikilink = pd.read_csv('assets\Wikidata_Ultimate_Source\links.csv')

In [31]:
wikilink.head(4)

,Unnamed: 0,Name,Wikidata_link,image
0,0,Abbott's Booby,https://www.wikidata.org/wiki/Q839854,NaN
1,0,Abbott's Starling,https://www.wikidata.org/wiki/Q27075615,NaN
2,0,Abd al Kuri Sparrow,https://www.wikidata.org/wiki/Q3056159,NaN
3,0,Abdim's Stork,https://www.wikidata.org/wiki/Q308199,NaN


In [44]:
import numpy as np 
wikilink['image'] = np.nan

In [48]:
def get_img(link):
    if not pd.isna(link):
        try:
            entity = client.get(link.split('/')[-1], load=True)
            image_prop = client.get('P18')
            image = entity[image_prop]
            t = image.title
            if 'File:' in t:
                imagename = t.strip().split(':')[-1]
                return imagename
            return np.nan
        except:
            return np.nan
    return np.nan
# get_img('https://www.wikidata.org/wiki/Q8061335')

In [49]:
from tqdm import tqdm
wikilink.reset_index(inplace=True)

In [50]:
for i in tqdm(range(wikilink.shape[0])):
    wikilink.loc[i,'image'] = get_img(wikilink.loc[i,'Wikidata_link'])

 44%|████▍     | 5540/12657 [4:28:01<2:48:51,  1.42s/it]     

Error: Canceled future for execute_request message before replies were done

In [ ]:
wikilink.to_pickle('assets\Wikidata_Ultimate_Source\link_image.pickle')

In [ ]:
wikilink.to_csv('assets\Wikidata_Ultimate_Source\link_image.csv')
wikilink.to_excel('assets\Wikidata_Ultimate_Source\link_image.excel')

In [3]:
images = pd.read_pickle('assets/Wikidata_Ultimate_Source/assets/Wikidata_Ultimate_Source/link_image.pkl')

In [4]:
images.head(5)

,index,Unnamed: 0,Name,Wikidata_link,image
0,0,0,Abbott's Booby,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg
1,1,0,Abbott's Starling,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg
2,2,0,Abd al Kuri Sparrow,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg
3,3,0,Abdim's Stork,https://www.wikidata.org/wiki/Q308199,Ciconia abdimii -London Zoo-8a.jpg
4,4,0,Aberdare Cisticola,https://www.wikidata.org/wiki/Q1923584,NaN


In [9]:
images.drop(columns=['index','Unnamed: 0'],inplace =True)
images.describe()

,Name,Wikidata_link,image
count,12657,10197,8925
unique,12657,9822,8562
top,Abbott's Booby,https://www.wikidata.org/wiki/Q21416306,"Myiagra ferrocyanea ferrocyanea, Myiagra cervi..."
freq,1,3,3


In [10]:
birds = pd.read_pickle('data\Birds.pkl')
birds.head(5)

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,...,Wiki_Description,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link
0,Abbott's Babbler,అబోట్ యొక్క బాబ్లెర్,Malacocincla abbotti,మాలాకోసిన్లా అబ్బోటి,Abbott's Babbler,అబోట్ యొక్క బాబ్లెర్,NaN,NaN,Blyth,బ్లిత్,...,Abbott's babbler (Malacocincla abbotti) is a s...,అబోట్ యొక్క బాబ్లర్ (మలాకోసింక్లా అబోట్టి) అనే...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22715811/9...
1,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,...,Abbott's booby (Papasula abbotti) is an endang...,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...
2,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,...,The Abbott's starling (Poeoptera femoralis) is...,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...
3,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,...,Abd al Kuri (Arabic: عبد الكوري‎) is a rocky i...,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...
4,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,Ciconia abdimii,సికోనియా అబ్దిమిఐ,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,"Sphenorrhynchus abdimii, Sphenorhynchus abdimii","స్ఫెనోరిన్చుస్ అబ్డిమి, స్ఫెన్నోరిచ్యున్చుస్ అ...",Lichtenstein,లిచ్టెన్ స్టెయిన్,...,"Abdim's stork (Ciconia abdimii), also known as...","అబ్దిమ్ కొంగ (సికోనియా అబ్దిమి), తెల్ల బొడ్డు ...",https://ebird.org/species/abdsto1,"A medium-sized, dark stork with a white belly....","తెల్లటి బొడ్డుతో మధ్యస్థ-పరిమాణ, ముదురు కొంగ. ...",https://eol.org/pages/45515774,Ciconia abdimii (Abdim's Stork) is a species o...,సికోనియా అబ్దిమి (అబ్డిమ్ కొంగ) అనేది సికోనిడే...,https://dibird.com/species/abdims-stork/,https://www.iucnredlist.org/species/22697673/9...


In [12]:
data = pd.merge(birds,images,left_on='Bird Original Name',right_on='Name')

In [14]:
row_dict = {}
print(list(data.columns))

['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telu

In [15]:
temp = data[['image','Image']]
temp.describe()

,image,Image
count,8925,10523
unique,8562,6696
top,"Myiagra ferrocyanea ferrocyanea, Myiagra cervi...",https://upload.wikimedia.org/wikipedia/commons...
freq,3,995


In [16]:
data = data.drop(columns=['Image','References','Wiki_Description','Name'])

In [19]:
data

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,...,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image
0,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg
1,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg
2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg
3,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,Ciconia abdimii,సికోనియా అబ్దిమిఐ,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,"Sphenorrhynchus abdimii, Sphenorhynchus abdimii","స్ఫెనోరిన్చుస్ అబ్డిమి, స్ఫెన్నోరిచ్యున్చుస్ అ...",Lichtenstein,లిచ్టెన్ స్టెయిన్,...,https://ebird.org/species/abdsto1,"A medium-sized, dark stork with a white belly....","తెల్లటి బొడ్డుతో మధ్యస్థ-పరిమాణ, ముదురు కొంగ. ...",https://eol.org/pages/45515774,Ciconia abdimii (Abdim's Stork) is a species o...,సికోనియా అబ్దిమి (అబ్డిమ్ కొంగ) అనేది సికోనిడే...,https://dibird.com/species/abdims-stork/,https://www.iucnredlist.org/species/22697673/9...,https://www.wikidata.org/wiki/Q308199,Ciconia abdimii -London Zoo-8a.jpg
4,Aberdare Cisticola,అబెర్డేర్ సిస్టోలా,Cisticola aberdare,సిస్కోలా అబెర్డేర్,Aberdare Cisticola,అబెర్డేర్ సిస్టోలా,Cisticola robusta aberdare,సిస్కోలా రొస్టా అబెర్డేర్,Lynes,లయన్స్,...,https://ebird.org/species/abecis1,A large and relatively colorful cisticola ende...,కెన్యాకు చెందిన పెద్ద మరియు సాపేక్షంగా రంగురంగ...,https://eol.org/pages/45514495,Cisticola aberdare (Aberdare Cisticola) is a s...,సిస్టికోలా అబెర్‌డేర్ (అబెర్‌డేర్ సిస్టికోలా) ...,https://dibird.com/species/aberdare-cisticola/,https://www.iucnredlist.org/species/22713436/1...,https://www.wikidata.org/wiki/Q1923584,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12652,Yucatan Rough-winged Swallow,యుకాటన్ రఫ్-వింగ్ స్వో,Stelgidopteryx ridgwayi,స్టెల్గిడోపోటెక్స్ రిడ్జ్వేయి,"Yucatan Rough-winged Swallow, Ridgway's Rough-...","యుకాటన్ రఫ్-రెక్కల స్వో, రిడ్వేయా యొక్క రఫ్-రె...",NaN,NaN,Nelson,నెల్సన్,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22712157/1...,NaN,NaN
12653,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,Suthora ricketti,సుథోరా రికెట్టి,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,NaN,NaN,Rothschild,రోత్స్కిల్డ్,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103887001/...,https://www.wikidata.org/wiki/Q8061335,NaN
12654,Zambian Barbet,జాంబియన్ బార్బెట్,Ly

In [20]:
data[['Wikidata_link']]

,Wikidata_link
0,https://www.wikidata.org/wiki/Q839854
1,https://www.wikidata.org/wiki/Q27075615
2,https://www.wikidata.org/wiki/Q3056159
3,https://www.wikidata.org/wiki/Q308199
4,https://www.wikidata.org/wiki/Q1923584
...,...
12652,NaN
12653,https://www.wikidata.org/wiki/Q8061335
12654,https://www.wikidata.org/wiki/Q1261209
12655,https://www.wikidata.org/wiki/Q31874195


In [21]:
row_dict = {}
print(list(data.columns))

['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telu

In [22]:
cols = ['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telugu', 'Life Span', 'Life Span Telugu', 'altitude', 'altitude Telugu', 'visual system', 'visual system Telugu', 'auditory system', 'auditory system Telugu', 'hearing range', 'hearing range Telugu', 'hearing threshold', 'hearing threshold Telugu', 'locomotion', 'locomotion Telugu', 'mineralized skeleton contains', 'mineralized skeleton contains Telugu', 'body mass', 'body mass Telugu', 'Wiki_Link', 'Wiki_Description Telugu', 'Ebird_Link', 'Ebird_Description', 'Ebird_Description Telugu', 'Eol_Link', 'Eol_Description', 'Eol_Description Telugu', 'Dibird_Link', 'IUCN_link', 'Wikidata_link', 'image']
len(cols)

82

In [23]:
for i in cols:row_dict[i] = ''
row_dict

{'Bird Original Name': '',
 'Bird Original Name Telugu': '',
 'Scientific Name': '',
 'Scientific Name Telugu': '',
 'Synonyms': '',
 'Synonyms Telugu': '',
 'Old latin name for bird': '',
 'Old latin name for bird Telugu': '',
 'Scientist': '',
 'Scientist Telugu': '',
 'founded': '',
 'founded Telugu': '',
 'Kingdom': '',
 'Kingdom Telugu': '',
 'Subkingdom': '',
 'Subkingdom Telugu': '',
 'Infrakingdom': '',
 'Infrakingdom Telugu': '',
 'Phylum': '',
 'Phylum Telugu': '',
 'Subphylum': '',
 'Subphylum Telugu': '',
 'Infraphylum': '',
 'Infraphylum Telugu': '',
 'Superclass': '',
 'Superclass Telugu': '',
 'Class': '',
 'Class Telugu': '',
 'Order': '',
 'Order Telugu': '',
 'Family': '',
 'Family Telugu': '',
 'Genus': '',
 'Genus Telugu': '',
 'Species': '',
 'Species Telugu': '',
 'Conservation status': '',
 'Conservation status Telugu': '',
 'Habitat': '',
 'Habitat Telugu': '',
 'Country': '',
 'Country Telugu': '',
 'geographic distribution includes': '',
 'geographic distribut